In [ ]:
#@title ENV Preparation/Hazırlıklar

!apt install pciutils neofetch nano
!git clone https://github.com/comfyanonymous/ComfyUI.git
%cd /content/ComfyUI/
!pip install -r requirements.txt
%cd ./cusom_nodes/
!git clone https://github.com/kijai/ComfyUI-HunyuanVideoWrapper.git
!pip install -r ComfyUI-HunyuanVideoWrapper/requirements.txt
%cd /content/ComfyUI/
!wget -c -nc https://huggingface.co/Comfy-Org/HunyuanVideo_repackaged/resolve/main/split_files/text_encoders/llava_llama3_fp8_scaled.safetensors -P ./models/text_encoders/
!wget -c -nc https://huggingface.co/Comfy-Org/HunyuanVideo_repackaged/resolve/main/split_files/text_encoders/clip_l.safetensors -P ./models/text_encoders/
!wget -c -nc https://huggingface.co/Comfy-Org/HunyuanVideo_repackaged/resolve/main/split_files/clip_vision/llava_llama3_vision.safetensors -P ./models/clip_vision/
!wget -c -nc https://huggingface.co/Comfy-Org/HunyuanVideo_repackaged/resolve/main/split_files/vae/hunyuan_video_vae_bf16.safetensors -P ./models/vae/
!wget -c -nc https://huggingface.co/Kijai/HunyuanVideo_comfy/resolve/main/hunyuan_video_vae_fp32.safetensors -P ./models/vae/
!wget -c -nc https://huggingface.co/Kijai/HunyuanVideo_comfy/resolve/main/hunyuan_video_I2V_720_fixed_bf16.safetensors -P ./models/diffusion_models/
!wget -c -nc https://huggingface.co/Kijai/HunyuanVideo_comfy/resolve/main/hunyuan_video_I2V_720_fixed_fp8_e4m3fn.safetensors -P ./models/diffusion_models/
!wget -c -nc https://huggingface.co/Kijai/HunyuanVideo_comfy/resolve/main/hunyuan_video_I2V-Q3_K_S.gguf -P ./models/diffusion_models/
!git clone https://github.com/ltdrdata/ComfyUI-Manager ./custom_nodes/comfyui-manager
!pip install xformers


In [ ]:
#@title Cloudflared ile tunneling yaparak açma
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    #print(l, end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title iframe ile açma

import threading
def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  from google.colab import output
  output.serve_kernel_port_as_iframe(port, height=1024)
  print("to open it in a window you can open this link here:")
  output.serve_kernel_port_as_window(port)

threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server